In [ ]:
!pip install -q parsl transformers accelerate

import math, json, time, random
from typing import List, Dict, Any
import parsl
from parsl.config import Config
from parsl.executors import ThreadPoolExecutor
from parsl import python_app

parsl.load(Config(executors=[ThreadPoolExecutor(label="local", max_threads=8)]))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.4/559.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

INFO:parsl.dataflow.dflow:Starting DataFlowKernel with config
Config(
    app_cache=True, 
    checkpoint_files=None, 
    checkpoint_mode=None, 
    checkpoint_period=None, 
    dependency_resolver=None, 
    executors=(ThreadPoolExecutor(
        label='local', 
        max_threads=8, 
        remote_monitoring_radio=<parsl.monitoring.radios.multiprocessing.MultiprocessingQueueRadio object at 0x7e8c009b0d90>, 
        storage_access=None, 
        thread_name_prefix='', 
        working_dir=None
    ),), 
    exit_mode='cleanup', 
    garbage_collect=True, 
    initialize_logging=True, 
    internal_tasks_max_threads=10, 
    max_idletime=120.0, 
    monitoring=None, 
    project_name=None, 
    retries=0, 
    retry_handler=None, 
    run_dir='runinfo', 
    std_autopath=None, 
    strategy='simple', 
    strategy_period=5, 
    usage_tracking=0
)
INFO:parsl.dataflow.dflow:Parsl version: 2025.08.11
DEBUG:parsl.usage_tracking.usage:Tracking level: 0
INFO:parsl.dataflow.dflow:Run id i

In [ ]:
@python_app
def calc_fibonacci(n: int) -> Dict[str, Any]:
    def fib(k):
        a, b = 0, 1
        for _ in range(k): a, b = b, a + b
        return a
    t0 = time.time(); val = fib(n); dt = time.time() - t0
    return {"task": "fibonacci", "n": n, "value": val, "secs": round(dt, 4)}

@python_app
def count_primes(limit: int) -> Dict[str, Any]:
    sieve = [True]*(limit+1); sieve[0:2] = [False, False]
    for i in range(2, int(limit**0.5)+1):
        if sieve[i]:
            step = i
            sieve[i*i:limit+1:step] = [False]*(((limit - i*i)//step)+1)
    primes = sum(sieve)
    return {"task": "count_primes", "limit": limit, "count": primes}

@python_app
def extract_keywords(text: str, k: int = 8) -> Dict[str, Any]:
    import re, collections
    words = [w.lower() for w in re.findall(r"[a-zA-Z][a-zA-Z0-9\-]+", text)]
    stop = set("the a an and or to of is are was were be been in on for with as by from at this that it its if then else not no".split())
    cand = [w for w in words if w not in stop and len(w) > 3]
    freq = collections.Counter(cand)
    scored = sorted(freq.items(), key=lambda x: (x[1], len(x[0])), reverse=True)[:k]
    return {"task":"keywords","keywords":[w for w,_ in scored]}

@python_app
def simulate_tool(name: str, payload: Dict[str, Any]) -> Dict[str, Any]:
    time.sleep(0.3 + random.random()*0.5)
    return {"task": name, "payload": payload, "status": "ok", "timestamp": time.time()}

In [ ]:
def tiny_llm_summary(bullets: List[str]) -> str:
    from transformers import pipeline
    gen = pipeline("text-generation", model="sshleifer/tiny-gpt2")
    prompt = "Summarize these agent results clearly:\n- " + "\n- ".join(bullets) + "\nConclusion:"
    out = gen(prompt, max_length=160, do_sample=False)[0]["generated_text"]
    return out.split("Conclusion:", 1)[-1].strip()

In [ ]:
def plan(user_goal: str) -> List[Dict[str, Any]]:
    intents = []
    if "fibonacci" in user_goal.lower():
        intents.append({"tool":"calc_fibonacci", "args":{"n":35}})
    if "primes" in user_goal.lower():
        intents.append({"tool":"count_primes", "args":{"limit":100_000}})
    intents += [
        {"tool":"simulate_tool", "args":{"name":"vector_db_search","payload":{"q":user_goal}}},
        {"tool":"simulate_tool", "args":{"name":"metrics_fetch","payload":{"kpi":"latency_ms"}}},
        {"tool":"extract_keywords", "args":{"text":user_goal}}
    ]
    return intents

In [ ]:
def run_agent(user_goal: str) -> Dict[str, Any]:
    tasks = plan(user_goal)
    futures = []
    for t in tasks:
        if t["tool"]=="calc_fibonacci": futures.append(calc_fibonacci(**t["args"]))
        elif t["tool"]=="count_primes": futures.append(count_primes(**t["args"]))
        elif t["tool"]=="extract_keywords": futures.append(extract_keywords(**t["args"]))
        elif t["tool"]=="simulate_tool": futures.append(simulate_tool(**t["args"]))
    raw = [f.result() for f in futures]

    bullets = []
    for r in raw:
        if r["task"]=="fibonacci":
            bullets.append(f"Fibonacci({r['n']}) = {r['value']} computed in {r['secs']}s.")
        elif r["task"]=="count_primes":
            bullets.append(f"{r['count']} primes found ≤ {r['limit']}.")
        elif r["task"]=="keywords":
            bullets.append("Top keywords: " + ", ".join(r["keywords"]))
        else:
            bullets.append(f"Tool {r['task']} responded with status={r['status']}.")

    narrative = tiny_llm_summary(bullets)
    return {"goal": user_goal, "bullets": bullets, "summary": narrative, "raw": raw}

In [ ]:
if __name__ == "__main__":
    goal = ("Analyze fibonacci(35) performance, count primes under 100k, "
            "and prepare a concise executive summary highlighting insights for planning.")
    result = run_agent(goal)
    print("\n=== Agent Bullets ===")
    for b in result["bullets"]: print("•", b)
    print("\n=== LLM Summary ===\n", result["summary"])
    print("\n=== Raw JSON ===\n", json.dumps(result["raw"], indent=2)[:800], "...")

DEBUG:parsl.dataflow.dflow:Task 0 will be sent to executor local
DEBUG:parsl.dataflow.dflow:Adding output dependencies
DEBUG:parsl.dataflow.dflow:Added output dependencies
DEBUG:parsl.dataflow.dflow:Gathering dependencies
DEBUG:parsl.dataflow.dflow:Gathered dependencies
INFO:parsl.dataflow.dflow:Task 0 submitted for App calc_fibonacci, not waiting on any dependency
DEBUG:parsl.dataflow.dflow:Task 0 set to pending state with AppFuture: <AppFuture at 0x7e8bf33cf7d0 state=pending>
DEBUG:parsl.dataflow.memoization:Task 0 will not be memoized
DEBUG:parsl.dataflow.dflow:Task 1 will be sent to executor local
DEBUG:parsl.dataflow.dflow:Adding output dependencies
DEBUG:parsl.dataflow.dflow:Added output dependencies
INFO:parsl.dataflow.dflow:Parsl task 0 try 0 launched on executor local
DEBUG:parsl.dataflow.dflow:Gathering dependencies
INFO:parsl.dataflow.dflow:Standard out for task 0 will not be redirected.
INFO:parsl.dataflow.dflow:Standard error for task 0 will not be redirected.
INFO:parsl.d

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

DEBUG:parsl.jobs.strategy:general strategy starting with strategy_type simple for 0 executors
DEBUG:parsl.process_loggers:Normal ending for _general_strategy on thread JobStatusPoller-Timer-Thread-139139544589456


pytorch_model.bin:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=160) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Agent Bullets ===
• Fibonacci(35) = 9227465 computed in 0.0s.
• 9592 primes found ≤ 100000.
• Tool vector_db_search responded with status=ok.
• Tool metrics_fetch responded with status=ok.
• Top keywords: highlighting, performance, fibonacci, executive, insights, planning, analyze, prepare

=== LLM Summary ===
 stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs stairs sta